In [1]:

!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37
Suggested packages:
  mpich-doc
The following NEW packages will be installed:
  hwloc-nox libmpich-dev libmpich12 libslurm37 mpich
0 upgraded, 5 newly installed, 0 to remove and 38 not upgraded.
Need to get 14.2 MB of archives.
After this operation, 102 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libslurm37 amd64 21.08.5-2ubuntu1 [542 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 hwloc-nox amd64 2.7.0-2ubuntu1 [205 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich12 amd64 4.0-3 [5,866 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 mpich amd64 4.0-3 [197 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmpich-dev amd64 4.0-3 [7,375 kB]
Fetched 14.2 MB in 

In [2]:

!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.5 MB/s eta 0:00:00


In [3]:

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()


local_n = N // size


A = None
if rank == 0:
    A = np.arange(1, N + 1, dtype=np.float64)


local_A = np.zeros(local_n, dtype=np.float64)

comm.Scatter([A, MPI.DOUBLE], [local_A, MPI.DOUBLE], root=0)

local_sum = np.sum(local_A)

global_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

if rank == 0:
    expected = N * (N + 1) / 2
    print(f"Total Sum = {global_sum:.0f}")
    print(f"Expected  = {expected:.0f}")
    print(f"Difference = {abs(global_sum - expected):.5f}")


Total Sum = 50000005000000
Expected  = 50000005000000
Difference = 0.00000


In [4]:
!mpirun -np 4 python3 distributed_sum.py

--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpirun as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


In [6]:

from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 10_000_000
local_n = N // size


A = None
if rank == 0:
    A = np.arange(1, N + 1, dtype=np.float64)
else:
    A = None


local_A = np.zeros(local_n, dtype=np.float64)
comm.Scatter([A, MPI.DOUBLE], [local_A, MPI.DOUBLE], root=0)

local_sum = np.sum(local_A)

comm.Barrier()
start = MPI.Wtime()

global_sum = comm.allreduce(local_sum, op=MPI.SUM)
average = global_sum / N

comm.Barrier()
end = MPI.Wtime()

if rank == 0:
    expected = N * (N + 1) / 2
    print(f"\nMPI_Allreduce Results:")
    print(f"Total Sum  = {global_sum:.0f}")
    print(f"Average    = {average:.4f}")
    print(f"Expected   = {expected:.0f}")
    print(f"Time Taken = {end - start:.5f} sec")


MPI_Allreduce Results:
Total Sum  = 50000005000000
Average    = 5000000.5000
Expected   = 50000005000000
Time Taken = 0.00037 sec


In [7]:
!mpirun -np 4 python3 sum_avg_allreduce.py

--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpirun as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


In [8]:

import numpy as np
import time

N = 10_000_000
A = np.arange(1, N + 1, dtype=np.float64)

start = time.time()
total_sum = np.sum(A)
end = time.time()

expected = N * (N +1)/2
print(f"Serial sum={total_sum:.0f}")
print(f"Expected={expected:.0f}")
print(f"Time taken={end - start:.5f}sec")

Serial Sum = 50000005000000
Expected   = 50000005000000
Time Taken = 0.01073 sec
